## 문장 임베딩 활용

In [1]:
# 문장 유사도 계산
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
words = ['학교', '공부', '게임', '자전거']

dense_embedded = model_sentence.encode(words)
dense_embedded
len(dense_embedded[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

768

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(dense_embedded)


array([[0.9999999 , 0.59507424, 0.30069774, 0.20964187],
       [0.59507424, 0.9999998 , 0.41403902, 0.16175744],
       [0.30069774, 0.41403902, 1.0000001 , 0.29132563],
       [0.20964187, 0.16175744, 0.29132563, 1.0000001 ]], dtype=float32)

In [4]:
# sentences = [
#     "그 식당은 파리를 날린다",
#     "그 식당에는 손님이 없다",
#     # "그 식당에서는 드론을 날린다",
#     # "파리가 식당에 날아다닌다",
#     # "학생이 자전거타고 학교에 간다",
#     "오늘 날씨가 정말 좋네요.",
#     "오늘 날씨가 맑고 기분이 좋아요.",
#     "인공지능을 활용하면 업무 효율을 높일 수 있다.",
#     "AI 기술을 적용하면 생산성이 향상된다.",
#     "강아지는 사람에게 충성심이 강한 동물이다.",
#     "주식시장은 변동성이 크고 예측하기 어렵다."
# ]
sentences = [
    "우리 팀은 이번 시즌 5연패를 기록하며 순위가 하락했다.",  # (1) 연속 패배
    "그 선수는 연패를 끊기 위해 전략을 변경했다.",  # (1) 연속 패배
    "챔피언십 경기에서 연패를 당한 팀은 다음 시즌을 대비하고 있다.",  # (1) 연속 패배
    "이 건물의 기둥은 정교한 연패 문양으로 장식되어 있다.",  # (2) 연속적인 무늬
    "고대 유적에서는 독특한 연패 형식의 조각을 볼 수 있다.",  # (2) 연속적인 무늬
    "그 도자기의 가장자리는 연패 무늬로 아름답게 꾸며져 있었다.",  # (2) 연속적인 무늬
    "그 선수는 대회에서 3년 연속 우승하며 연패의 위업을 달성했다.",  # (3) 연속 재패
    "우리 팀은 세계 챔피언십에서 2연패를 기록하며 최강의 자리를 유지했다."  # (3) 연속 재패
]
sentence_embeddings = model_sentence.encode(sentences)

similarities = model_sentence.similarity(sentence_embeddings, sentence_embeddings)
# similarities = model_sentence.similarity([sentence_embeddings[-4]], sentence_embeddings)
similarities

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[1.0000, 0.4483, 0.6003, 0.1000, 0.1866, 0.0172, 0.3438, 0.5940],
        [0.4483, 1.0000, 0.4939, 0.0731, 0.1611, 0.0337, 0.5638, 0.4341],
        [0.6003, 0.4939, 1.0000, 0.1940, 0.2312, 0.0791, 0.4004, 0.6605],
        [0.1000, 0.0731, 0.1940, 1.0000, 0.6963, 0.6681, 0.1861, 0.2104],
        [0.1866, 0.1611, 0.2312, 0.6963, 1.0000, 0.6514, 0.1672, 0.1984],
        [0.0172, 0.0337, 0.0791, 0.6681, 0.6514, 1.0000, 0.1303, 0.1460],
        [0.3438, 0.5638, 0.4004, 0.1861, 0.1672, 0.1303, 1.0000, 0.6329],
        [0.5940, 0.4341, 0.6605, 0.2104, 0.1984, 0.1460, 0.6329, 1.0000]])